In [ ]:
from datetime import datetime
from netCDF4 import Dataset as ds
import numpy as np
import dateutil as du 
import parser
import pytz
import dateutil.parser 
import pandas as pd
from pandas import DataFrame
import csv
import glob, os
from os.path import expanduser
from pathlib import Path
import xarray as xr 

In [ ]:
df = pd.read_csv('df_adjusted_date.csv')

In [ ]:
# Change days to using adjusted date
df['adjusted_date'] = df['adjusted_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df['DAY'] = df['adjusted_date'].dt.day
df['MONTH'] = df['adjusted_date'].dt.month
df['YEAR'] = df['adjusted_date'].dt.year

In [ ]:
# insert '0' and convert to string
df['DAY'] = df['DAY'].apply(lambda x: int(float(x)))
df['DAY'] = df['DAY'].apply('{0:0>2}'.format)
df['MONTH'] = df['MONTH'].apply(lambda x: int(float(x)))
df['MONTH'] = df['MONTH'].apply('{0:0>2}'.format)

In [ ]:
# Function to slice the filename with .nc extention, this will be used to open the files in directory
def file_info(ncfilename):
    with ds(ncfilename,'r') as ncdat:
        #string = str(ncfilename)
        #filename = string[-16:] 
        array = xr.open_dataset(ncfilename) # open_dataset works fine with either full path or filename.nc 
        return array  
def array_sel(array):
    dsloc = array.sel(lon=lon,lat=lat, method='nearest') 
    val_arr = dsloc['hotspot']
    val_arr = val_arr.rolling(time=63).sum()
    val_arr = val_arr.sel(time=time.strftime("%Y-%m-%d")) # slice to a single given date
    return val_arr 

In [ ]:
# Extract arrays per row and put them in a list
home = Path(expanduser("~"))
x = []
for row in df.itertuples():
        lat = row.lat
        lon = row.lon
        time = row.full_date
        path = home / Path("C:/Users/bmped/Documents/SST_HS/HS_1985-2018/HS_1985-2018_0.nc") 
        array = file_info(path)
        cf = array_sel(array)
        x.append(cf)


In [ ]:
# # create a list for sst values # create lists for coordinates (time) 
DHW_9 = [item.values for item in x]   # get array of values; analysed sst in this case
df['DHW_9'] = DHW_9
df.to_csv('DHW_9_adjusted_date.csv')